## Custom class implementation using Keras Api :

- source : tensorflow 

In [1]:
import os
import tensorflow as tf 
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from warnings import filterwarnings
filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
tf.__version__

'2.6.0'

In [3]:
print(len(tf.config.list_physical_devices('GPU')),'\n')

1 



## Load and perpare MNIST dataset.

In [4]:
mnist = tf.keras.datasets.mnist
(trainInputs, trainTarget), (testInputs, testTarget) = mnist.load_data()

print(trainInputs.shape, trainTarget.shape)
print(testInputs.shape, testTarget.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [5]:
trainInputs, testInputs = trainInputs / 255.0, testInputs / 255.0

# Add a channels dimension :
trainInputs = trainInputs[..., tf.newaxis].astype("float32")
testInputs = testInputs[..., tf.newaxis].astype("float32")

In [6]:
print(trainInputs.shape, testInputs.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


- tf.data to shuffle the dataset :

In [7]:
trainDs = tf.data.Dataset.from_tensor_slices((trainInputs, trainTarget)).shuffle(10000).batch(32)

testDs = tf.data.Dataset.from_tensor_slices((testInputs, testTarget)).batch(32)

## Build custom model using Keras model subclassing API :

In [8]:
class model(Model) :
    def __init__(self) :
        super(model, self).__init__()
        self.conv1 = Conv2D(32, 2, activation = 'relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation = 'relu')
        self.d2 = Dense(10)

    def call(self, x) :
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model :
Model = model()

- Choose an optimizer and loss function for training:

In [9]:
lossObj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

optimizer = tf.keras.optimizers.Adam()

- Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [10]:
trainLoss = tf.keras.metrics.Mean(name = 'trainLoss')
trainAccuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'trainAccuracy')

testLoss = tf.keras.metrics.Mean(name = 'testLoss')
testAccuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'testAccuracy')


In [11]:
# -------------- train the model using gradient descent --------------

@tf.function
def trainStep(images, labels) :
    with tf.GradientTape() as tape :
        predictions = Model(images, training = True)
        loss = lossObj(labels, predictions)
    gradients = tape.gradient(loss, Model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, Model.trainable_variables))

    trainLoss(loss)
    trainAccuracy(labels, predictions)

## Test the model :

In [12]:
@tf.function
def testStep(images, labels) :
    predictions = Model(images, training = False)
    tLoss = lossObj(labels, predictions)

    testLoss(tLoss)
    testAccuracy(labels, predictions)

In [13]:
EPOCHS = 5

for epoch in range(EPOCHS) :
    # Reset the metrics at the start of the next episode 
    trainLoss.reset_states()
    trainAccuracy.reset_states()
    testLoss.reset_states()
    testAccuracy.reset_states()

    for images, labels in trainDs :
        trainStep(images, labels)
    
    for testImages, testLabels in testDs :
        testStep(testImages, testLabels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss : {trainLoss.result()}, '
        f'Accuracy : {trainAccuracy.result() * 100}, '
        f'Test Loss : {testLoss.result()}, '
        f'Test Accuracy : {testAccuracy.result() * 100}'
    )

Epoch 1, Loss : 0.17003782093524933, Accuracy : 94.9383316040039, Test Loss : 0.07792789489030838, Test Accuracy : 97.62999725341797
Epoch 2, Loss : 0.05009588226675987, Accuracy : 98.49333190917969, Test Loss : 0.06883905827999115, Test Accuracy : 97.75
Epoch 3, Loss : 0.025055253878235817, Accuracy : 99.26499938964844, Test Loss : 0.0640917420387268, Test Accuracy : 98.07999420166016
Epoch 4, Loss : 0.013801013119518757, Accuracy : 99.52832794189453, Test Loss : 0.06464999169111252, Test Accuracy : 98.16999816894531
Epoch 5, Loss : 0.010244538076221943, Accuracy : 99.6383285522461, Test Loss : 0.06668376922607422, Test Accuracy : 98.22999572753906
